In [ ]:
import os
os.chdir(r"c:\Users\rohan\Documents\Flutter Projects\saas-subscription-analytics")
os.getcwd()

In [ ]:
import duckdb
con = duckdb.connect("saas.db")

con.execute("""
CREATE TABLE accounts AS SELECT * FROM 'data/raw/ravenstack_accounts.csv';
CREATE TABLE subscriptions AS SELECT * FROM 'data/raw/ravenstack_subscriptions.csv';
CREATE TABLE churn_events AS SELECT * FROM 'data/raw/ravenstack_churn_events.csv';
CREATE TABLE feature_usage AS SELECT * FROM 'data/raw/ravenstack_feature_usage.csv';
CREATE TABLE support_tickets AS SELECT * FROM 'data/raw/ravenstack_support_tickets.csv';
""")

print(con.execute("SHOW TABLES;").fetchall())

In [ ]:
import duckdb

con = duckdb.connect("saas.db")

clean_files = [
    "sql/11_clean_accounts.sql",
    "sql/12_clean_subscriptions.sql",
    "sql/13_clean_churn_events.sql",
    "sql/14_clean_feature_usage.sql",
    "sql/15_clean_support_tickets.sql"
]

# Run each SQL script in the correct order
for path in clean_files:
    print(f"\nRunning: {path}")
    with open(path, "r") as f:
        sql = f.read()
    con.execute(sql)

print("\nAll cleaning scripts completed.\n")

# Export cleaned tables
exports = {
    "accounts_clean": "data/processed/accounts_cleaned.csv",
    "subscriptions_clean": "data/processed/subscriptions_cleaned.csv",
    "churn_events_clean": "data/processed/churn_events_cleaned.csv",
    "feature_usage_clean": "data/processed/feature_usage_cleaned.csv",
    "support_tickets_clean": "data/processed/support_tickets_cleaned.csv"
}

for table, path in exports.items():
    print(f"Exporting: {table} → {path}")
    con.execute(f"COPY {table} TO '{path}' WITH (HEADER TRUE);")

con.close()
print("\nDone!")


In [ ]:
import duckdb
con = duckdb.connect("saas.db")
con.execute("SHOW TABLES;").fetchall()


In [ ]:
# Force-close all DuckDB connections held by Python
import duckdb, gc

try:
    duckdb.default_connection.close()
except:
    pass

# force garbage collection to release any lingering locks
gc.collect()

print("All DuckDB connections closed.")
